In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler  
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

import statsmodels.api as sm
import statsmodels.formula.api as smf

https://github.com/mwaskom/seaborn-data
For others dataset

In [ ]:
df = sns.load_dataset("iris")

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.mean()

In [ ]:
df.std()

In [ ]:
df.std()/df.mean()

In [ ]:
stats.zscore(df.sepal_length)

In [ ]:
sns.pairplot(df, hue="species", height=2, aspect=1)

In [ ]:
df.hist()
plt.show()

In [ ]:
plt.figure(figsize = (10, 8))
plt.scatter(df.petal_length, df.petal_width);

# ML LR

In [ ]:
LRModel = LinearRegression(fit_intercept=True)

In [ ]:
LRModel

In [ ]:
RSCModel = RANSACRegressor()

In [ ]:
RSCModel

In [ ]:
X = df.petal_length.values.reshape(-1,1)

In [ ]:
y= df.petal_width.values

In [ ]:
LRModel.fit(X, y)

In [ ]:
LRModel.coef_

In [ ]:
LRModel.intercept_

In [ ]:
x_fit = np.linspace(0, 8)

In [ ]:
X_fit = x_fit.reshape(-1,1)

In [ ]:
X_fit

In [ ]:
y_fit = LRModel.predict(X_fit)

In [ ]:
y_fit

In [ ]:
plt.figure(figsize = (10, 8))
plt.scatter(df.petal_length.values, df.petal_width.values)
plt.plot(x_fit, y_fit);

# BOSTON Housong Project

In [ ]:
bost = pd.read_csv("housing.data", delim_whitespace=True, header= None)

In [ ]:
bost.columns = ["CrimePerCapita",
"ResidZone",
"IndusAcre",
"CHARiver",
"NOXConcent",
"RoomAvg",
"Age",
"Dist_Center",
"Axs_HW",
"Tax",
"Educ_ratio",
"Blck",
"LowStatus",
"MedValue"]

In [ ]:
bost.head()

In [ ]:
bost.describe()

In [ ]:
sns.pairplot(bost[["Educ_ratio", "Blck", "LowStatus","MedValue"]])

In [ ]:
bost.corr()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(bost.corr(), annot=True)

In [ ]:
ind= 5
# strong diff : 0 1 7 8 10 
# weak diff : 2 5 6 9 11 12
# exclude : 3 4
X = bost.iloc[:,ind].values.reshape(-1,1)
y = bost.MedValue.values

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)
LRModel.fit(X_train,y_train)
RSCModel.fit(X_train,y_train)
inliers = RSCModel.inlier_mask_
outliers = np.logical_not(inliers)

r_range = np.arange(bost.iloc[:,ind].min(),bost.iloc[:,ind].max(),1)
LR_r_y= LRModel.predict(r_range.reshape(-1,1))
RCS_r_y= RSCModel.predict(r_range.reshape(-1,1))
LR_train_y=LRModel.predict(X_train)
RSC_train_y=RSCModel.predict(X_train)
LR_test_y=LRModel.predict(X_test)
RSC_test_y=RSCModel.predict(X_test)

print("Coef LR: ",LRModel.coef_)
print("Intercept LR: ",LRModel.intercept_)
print("Coef RSC: ",RSCModel.estimator_.coef_)
print("Intercept RSC: ",RSCModel.estimator_.intercept_)
print("MSE LR Train: ",mean_squared_error(y_train, LR_train_y));
print("MSE RSC Train: ",mean_squared_error(y_train, RSC_train_y));
print("MSE LR Test: ",mean_squared_error(y_test, LR_test_y));
print("MSE RSC Test: ",mean_squared_error(y_test, RSC_test_y));
print("R² LR Train: ",r2_score(y_train, LR_train_y));
print("R² RSC Train: ",r2_score(y_train, RSC_train_y));
print("R² LR Test: ",r2_score(y_test, LR_test_y));
print("R² RSC Test: ",r2_score(y_test, RSC_test_y));

plt.figure(figsize=(12,10))
plt.scatter(X_train[inliers], y_train[inliers], c="black", marker="*", label="Inliers")
plt.scatter(X_train[outliers], y_train[outliers], c="yellow", marker="*", label="Ouliers")
plt.plot(r_range,RSC_y, color="red")
plt.plot(r_range,LR_y, color="blue")
plt.xlabel("Medium Price")
plt.ylabel(bost.columns[ind])
plt.legend()
plt.figure(figsize=(12,8))
plt.scatter(LR_train_y, LR_train_y - y_train, c='blue', marker='o', label='Training data')
plt.scatter(LR_test_y, LR_test_y - y_test, c='orange', marker='*', label='Test data')
plt.xlabel('LM Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
#plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='k')
#plt.xlim([-10, 50])
plt.show()
plt.figure(figsize=(12,8))
plt.scatter(RSC_train_y, RSC_train_y - y_train, c='blue', marker='o', label='Training data')
plt.scatter(RSC_test_y, RSC_test_y - y_test, c='orange', marker='*', label='Test data')
plt.xlabel('RSC Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.show()

In [ ]:
bost_const = sm.add_constant(bost.iloc[:,:-1])
bost_target = bost.iloc[:,-1:].values

In [ ]:
bost_target

In [ ]:
model = sm.OLS(bost_target, bost_const)
LR = model.fit()
LR.summary()

In [ ]:
fmodel = smf.ols(formula = 'MedValue ~ CrimePerCapita + ResidZone + IndusAcre + CHARiver + RoomAvg + Age + Dist_Center + Axs_HW + Tax + Educ_ratio + Blck + LowStatus',
                data=bost)
LRf = fmodel.fit()
LRf.summary()

In [ ]:
r2_score(bost_target, LRf.predict(bost))

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format
bost_corr = bost.corr()
bost_corr

In [ ]:
bost_corr[np.abs(bost_corr) <= 0.7] = 0
bost_corr
plt.figure(figsize=(16,10))
sns.heatmap(bost_corr, annot=True, cmap='YlGnBu')
plt.show()

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(bost.corr())
pd.Series(eigenvalues).sort_values()

In [ ]:
np.abs(pd.Series(eigenvectors[:,8])).sort_values(ascending=False)

In [ ]:
print(bost.columns[2], bost.columns[8], bost.columns[9])

In [ ]:
model = LinearRegression()
model.fit(bost.iloc[:,:-1],bost_target)
model.coef_

In [ ]:
scaler = StandardScaler()  
Stand_coef_linear_reg = make_pipeline(scaler, model)

In [ ]:
Stand_coef_linear_reg.fit(bost.iloc[:,:-1],bost_target)
Stand_coef_linear_reg.steps[1][1].coef_

In [ ]:
X_boston = bost['Dist_Center'].values
y_boston = bost['NOXConcent'].values

In [ ]:
lr = LinearRegression()
lr.fit(X_boston.reshape(-1, 1), y_boston)
model_pred = lr.predict(X_boston.reshape(-1,1))
plt.figure(figsize=(12,8))
plt.scatter(X_boston, y_boston);
plt.plot(X_boston, model_pred);
print("R^2 score = {:.2f}".format(r2_score(y_boston, model_pred)))


In [ ]:
poly_reg = PolynomialFeatures(degree=3)
X_poly_b = poly_reg.fit_transform(X_boston.reshape(-1, 1))
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly_b, y_boston)

In [ ]:
X_fit = np.arange(X_boston.min(), X_boston.max(), 1)[:, np.newaxis]

In [ ]:
y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_fit.reshape(-1,1)))

In [ ]:
plt.figure(figsize=(10,8));
plt.scatter(X_boston, y_boston);
plt.plot(X_fit, y_pred);
print("R^2 score = {:.2f}".format(r2_score(y_boston, 
                                          lin_reg_2.predict(X_poly_b))))

# Seaborn Visualisation

In [ ]:
sns.jointplot(x='LowStatus',y='MedValue',data=bost, kind='reg', height = 10)

In [ ]:
sns.regplot(X,y)